<a href="https://colab.research.google.com/github/alexnodejs/bandit/blob/main/HT_23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель



In [1]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/ratings.csv")

df.head()

,user_id,movie_id,rating,timestamp
0,172,94969,5.0,1396067836
1,172,98956,4.0,1396067879
2,176,73881,4.0,1499807147
3,221,1900,4.5,1288550866
4,333,33688,4.0,1412015122


In [5]:
!pip install -q surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(0.1, 4))

data = Dataset.load_from_df(df[["user_id", "movie_id", "rating"]], reader)
data

In [7]:

from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(0.5, 5))

data = Dataset.load_from_df(df[["user_id", "movie_id", "rating"]], reader)
data


In [8]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.7)

In [9]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.002,
               'reg_i': 0.002}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [10]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=60,
           )

algo.fit(trainset)

Estimating biases using als...


In [12]:
from surprise import KNNBasic

algo = KNNBasic(k=20,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [13]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 100)

algo.fit(trainset)

In [14]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.001,
               'reg_i': 0.001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

Estimating biases using als...
mae = 0.6729053532678994
mse = 0.8513153461017023
rmse= 0.9226675165527951
fcp = 0.6640999261499558


In [15]:
algo.predict(uid=2, iid=98)

Prediction(uid=2, iid=98, r_ui=None, est=3.4616945301087485, details={'was_impossible': False})

In [18]:
# Model init
models = {
    "SVD": SVD(),
    "BaselineOnly": BaselineOnly(),
    "KNNBasic": KNNBasic()
}


results = {}
for model_name, model in models.items():
    model.fit(trainset)
    predictions = model.test(testset)

    rmse = accuracy.rmse(predictions, verbose=False)
    mae = accuracy.mae(predictions, verbose=False)

    results[model_name] = {"RMSE": rmse, "MAE": mae}

for model_name, metrics in results.items():
    print(f"{model_name}: RMSE = {metrics['RMSE']:.2f}, MAE = {metrics['MAE']:.2f}")

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
SVD: RMSE = 0.98, MAE = 0.73
BaselineOnly: RMSE = 0.99, MAE = 0.75
KNNBasic: RMSE = 1.07, MAE = 0.81


In [17]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.5 MB/s eta 0:00:00


In [19]:
import optuna
def objective(trial):
    # param config
    n_factors = trial.suggest_int("n_factors", 20, 200)
    n_epochs = trial.suggest_int("n_epochs", 20, 100)
    lr_all = trial.suggest_float("lr_all", 0.001, 0.1, log=True)
    reg_all = trial.suggest_float("reg_all", 0.01, 0.1, log=True)

    # model
    model = SVD(n_factors=n_factors, n_epochs=n_epochs, lr_all=lr_all, reg_all=reg_all)

    # train and test
    model.fit(trainset)
    predictions = model.test(testset)

    #  RMSE as loss function
    return accuracy.rmse(predictions, verbose=False)

# Launch Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Best
print("best params:")
study.best_params

[I 2024-12-15 01:11:31,378] A new study created in memory with name: no-name-73d7cc59-7401-402a-b731-019e3a5d6fa1
[I 2024-12-15 01:11:31,965] Trial 0 finished with value: 0.9407399549532702 and parameters: {'n_factors': 69, 'n_epochs': 63, 'lr_all': 0.006361936223481266, 'reg_all': 0.048141644758913416}. Best is trial 0 with value: 0.9407399549532702.
[I 2024-12-15 01:11:32,367] Trial 1 finished with value: 0.9339209356718089 and parameters: {'n_factors': 43, 'n_epochs': 52, 'lr_all': 0.006108049228901128, 'reg_all': 0.039310858496186686}. Best is trial 1 with value: 0.9339209356718089.
[I 2024-12-15 01:11:33,222] Trial 2 finished with value: 0.9276858143136345 and parameters: {'n_factors': 98, 'n_epochs': 77, 'lr_all': 0.005633128180009679, 'reg_all': 0.0791438369425874}. Best is trial 2 with value: 0.9276858143136345.
[I 2024-12-15 01:11:33,507] Trial 3 finished with value: 0.9473388559633688 and parameters: {'n_factors': 45, 'n_epochs': 38, 'lr_all': 0.010845391891418426, 'reg_all':

best params:


{'n_factors': 125,
 'n_epochs': 81,
 'lr_all': 0.022812263042526208,
 'reg_all': 0.09978842842397392}

In [20]:
# Use better params
best_params = study.best_params
best_model = SVD(**best_params)

best_model.fit(trainset)
best_predictions = best_model.test(testset)

# estimation
final_rmse = accuracy.rmse(best_predictions)
final_mae = accuracy.mae(best_predictions)

print(f" RMSE: {final_rmse:.4f}")
print(f" MAE: {final_mae:.4f}")

RMSE: 0.9216
MAE:  0.6926
 RMSE: 0.9216
 MAE: 0.6926
